In [49]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/pytorch/2b-it/2/config.json
/kaggle/input/gemma/pytorch/2b-it/2/gemma-2b-it.ckpt
/kaggle/input/gemma/pytorch/2b-it/2/tokenizer.model
/kaggle/input/data-science-interview-q-and-a-treasury/dataset.csv


In [50]:
!pip install -q -U immutabledict sentencepiece 
!git clone https://github.com/google/gemma_pytorch.git

fatal: destination path 'gemma_pytorch' already exists and is not an empty directory.


In [51]:
import sys
sys.path.append('/kaggle/working/gemma_pytorch/')

In [52]:
from gemma.config import GemmaConfig, get_config_for_7b, get_config_for_2b, get_model_config
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
import os
import torch

In [53]:
# Load Model
VARIANT = "2b-it"
MACHINE = "cuda"
weights_dir = "/kaggle/input/gemma/pytorch/2b-it/2"

In [54]:
# model_config = get_model_config(VARIANT)
model_config = get_config_for_2b()
model_config.tokenizer = os.path.join(weights_dir, "tokenizer.model")

In [55]:
print(model_config)

GemmaConfig(vocab_size=256000, max_position_embeddings=8192, num_hidden_layers=18, num_attention_heads=8, num_key_value_heads=1, hidden_size=2048, intermediate_size=16384, head_dim=256, rms_norm_eps=1e-06, dtype='bfloat16', quant=False, tokenizer='/kaggle/input/gemma/pytorch/2b-it/2/tokenizer.model')


In [56]:
torch.set_default_dtype(model_config.get_dtype())

In [57]:
device = torch.device(MACHINE)


model = GemmaForCausalLM(model_config)
print("The model: \n", model)

ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
model.load_weights(ckpt_path)
model = model.to(device)

The model: 
 GemmaForCausalLM(
  (embedder): Embedding()
  (model): GemmaModel(
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (qkv_proj): Linear()
          (o_proj): Linear()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear()
          (up_proj): Linear()
          (down_proj): Linear()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (sampler): Sampler()
)


In [58]:
# Chat templates - use gemma formatting
USER_CHAT_TEMPLATE = "<start_of_turn> user \n{prompt} <end_of_turn> \n"
MODEL_CHAT_TEMPLATE = "<start_of_turn> model\n{prompt} <end_of_turn> \n"

prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt="What is a good place for travel in USA?"
    )
    + MODEL_CHAT_TEMPLATE.format(prompt="New Jersey")
    + USER_CHAT_TEMPLATE.format(prompt="What can I do in New Jersey?")
    + "<start_of_turn>model\n"
)
print('Chat prompt:\n', prompt)

Chat prompt:
 <start_of_turn> user 
What is a good place for travel in USA? <end_of_turn> 
<start_of_turn> model
New Jersey <end_of_turn> 
<start_of_turn> user 
What can I do in New Jersey? <end_of_turn> 
<start_of_turn>model



In [60]:
output = model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device=device,
    output_len=128,
)

print(output)

**Cities:**

* **New York City:** The Big Apple boasts iconic sights like the Statue of Liberty and the Empire State Building, along with a diverse range of neighborhoods, cultural attractions, and entertainment.
* **Philadelphia:** Experience colonial history, visit the Liberty Bell, and explore the charm of the Old City.
* **Newark:** Home to rich history, a thriving cultural scene, and a diverse population, Newark offers a unique travel experience.
* **Jersey City:** Discover world-class restaurants, shops, and waterfront attractions in a vibrant waterfront setting.

**Natural Attractions:**

* **Cape May National Seashore:**
